In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.copy()
X = X.drop("loan_status", axis=1)
X = pd.get_dummies(X, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)


# Create our target
y = df["loan_status"].ravel()
y[:5]

array(['low_risk', 'low_risk', 'low_risk', 'low_risk', 'low_risk'],
      dtype=object)

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [8]:
# Check the balance of our target values
df['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1)

In [10]:
# Creating StandardScaler instance
#scaler = StandardScaler()

In [11]:
# Fitting Standard Scaler
#X_scaler = scaler.fit(X_train)

In [12]:
# Scaling data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(random_state=78, n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [14]:
# Display the confusion matrix
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   68,    33],
       [ 2230, 14874]], dtype=int64)

In [15]:
# Calculating the confusion matrix
cm5 = confusion_matrix(y_test, y_pred)

#Create a dataframe from the confusion matrix
cm5_df = pd.DataFrame(
    cm5, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm5_df

,Predicted 0,Predicted 1
Actual 0,68,33
Actual 1,2230,14874


In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7714442339930907

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.67      0.87      0.06      0.77      0.57       101
   low_risk       1.00      0.87      0.67      0.93      0.77      0.60     17104

avg / total       0.99      0.87      0.67      0.92      0.77      0.60     17205



In [18]:
# we can modify our code above as follows:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.01254685, 0.03256861, 0.01596504, 0.01371015, 0.01634084,
       0.00436737, 0.00740707, 0.00900612, 0.00202146, 0.01642081,
       0.01014029, 0.01756416, 0.01529921, 0.05108634, 0.06086602,
       0.07617683, 0.04563747, 0.00751612, 0.        , 0.        ,
       0.06676442, 0.00054032, 0.        , 0.        , 0.004746  ,
       0.01060964, 0.00683278, 0.00586702, 0.00447242, 0.00771273,
       0.01826238, 0.0117112 , 0.01286308, 0.00466749, 0.00546951,
       0.01600427, 0.01161891, 0.01415055, 0.00910997, 0.0079165 ,
       0.00913121, 0.01040457, 0.014628  , 0.01269839, 0.0130623 ,
       0.00027317, 0.        , 0.01642772, 0.01097585, 0.01032197,
       0.00895394, 0.00517761, 0.01188853, 0.0169541 , 0.00201387,
       0.00759536, 0.00896733, 0.00707946, 0.00831438, 0.00948344,
       0.01193446, 0.01133328, 0.00731907, 0.00904002, 0.        ,
       0.        , 0.00090363, 0.00732877, 0.00666986, 0.00795251,
       0.00153873, 0.        , 0.01742768, 0.01142909, 0.01159

In [19]:
# List the features sorted in descending order by feature importance
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07617682714468305, 'total_rec_prncp'),
 (0.06676441981513385, 'last_pymnt_amnt'),
 (0.06086601515552552, 'total_pymnt_inv'),
 (0.0510863446388865, 'total_pymnt'),
 (0.04563746771901525, 'total_rec_int'),
 (0.032568608940337036, 'int_rate'),
 (0.019752009487465392, 'issue_d_Jan-2019'),
 (0.018262376507438144, 'mths_since_rcnt_il'),
 (0.017564158224272344, 'out_prncp'),
 (0.01742768450837416, 'tot_hi_cred_lim'),
 (0.016954095445970162, 'mths_since_recent_inq'),
 (0.016737672861306462, 'issue_d_Mar-2019'),
 (0.016427721063382344, 'mo_sin_old_il_acct'),
 (0.01642080940747151, 'revol_bal'),
 (0.01634084343213558, 'dti'),
 (0.016004268565643177, 'max_bal_bc'),
 (0.015965037919855693, 'installment'),
 (0.015299208563154058, 'out_prncp_inv'),
 (0.014627995142203967, 'avg_cur_bal'),
 (0.014150554790636012, 'total_rev_hi_lim'),
 (0.013710150608170216, 'annual_inc'),
 (0.013062304432328725, 'bc_util'),
 (0.012863080552324359, 'il_util'),
 (0.012698390361482986, 'bc_open_to_buy'),
 (0.01254685

### Easy Ensemble AdaBoost Classifier

In [20]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import EasyEnsembleClassifier

In [21]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
eec_model = EasyEnsembleClassifier(random_state=78, n_estimators=100)
eec_model.fit(X_train, y_train)
y_pred = eec_model.predict(X_test)

In [22]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE


In [23]:
# Calculated the balanced accuracy score
y_pred = eec_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   92,     9],
       [  901, 16203]], dtype=int64)

In [24]:
# Calculating the confusion matrix
cm6 = confusion_matrix(y_test, y_pred)

#Create a dataframe from the confusion matrix
cm6_df = pd.DataFrame(
    cm6, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm6_df

,Predicted 0,Predicted 1
Actual 0,92,9
Actual 1,901,16203


In [25]:
# Display the confusion matrix
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.9291066764534265

In [26]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.91      0.95      0.17      0.93      0.86       101
   low_risk       1.00      0.95      0.91      0.97      0.93      0.87     17104

avg / total       0.99      0.95      0.91      0.97      0.93      0.87     17205

